<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       GLM Fraud Detection with Python and Vantage BYOM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In recent years we have seen a massive increase in fraud attempts, making fraud detection imperative for Banking and Financial Institutions. Despite countless efforts and human supervision, hundreds of millions of dollars are lost due to fraud. Fraud can happen using various methods, i.e., stolen credit cards, misleading accounting, phishing emails, etc. Due to small cases in significant populations, fraud detection has become more and more challenging. Fortunately, with Teradata Vantage’s Bring Your Own Model capabilities, data scientists can use their preferred language, tools and platform to develop models to identify this fraud. Even in large scale operations, users have the guarantee that Vantage can scale to their needs and reduce fraud.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Identification of financial fraud in multiple accounts</li>
    <li>Pattern recognition of fraudulent versus normal transactions</li>
    <li>Reduction of money lost due to recovering fraudulent charges</li>
    <li>Improved customer satisfaction and reduction of customer churn</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ML and AI industry continues to innovate at an unprecedented rate.  Tools, technologies, and algorithms	 are being developed and improved in both the open source and commercial communities.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Unfortunately, many of these techniques haven’t matured to the point where they are readily deployable to a stable, mature operational environment.  Furthermore, many open-source techniques rely on fragile, manual enabling technologies.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>ClearScape Analytics Bring Your Own Model capabilities allow organizations to leverage third party and open-source models for scoring inside the Vantage Platform; providing enterprise-class scalability and operational stability for any number of users, applications, or volume of data.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Installing some dependencies</b>

In [ ]:
# %%capture
# # '%%capture' suppresses the display of installation steps of the following packages
# !pip install sklearn2pmml jdk4py --user

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# import statements
import os
import getpass
import warnings
warnings.filterwarnings("ignore")

from teradataml import *
from sklearn.linear_model import LogisticRegression
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION

configure.byom_install_location = 'mldb'
display.max_rows = 5

# Set java path
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA_HOME)
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA)[:-5]

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=Financial_Fraud_Detection_BYOM_Python.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('demo_glm_fraud_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('demo_glm_fraud_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Read the data from Vantage as a teradaml Dataframe</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We loaded the data from <a href = 'https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data'>https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data</a> into Vantage in a table named "transaction_data". We checked the data size and printed sample rows: 63k rows and 12 columns.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</p>

In [ ]:
# Get data as a teradataml dataframe
txn_data = DataFrame(in_schema('DEMO_GLM_Fraud','transaction_data'))
print("Shape of the data: ", txn_data.shape)
txn_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this simulated scenario, deceptive agents engage in transactions with the objective of taking control of customers' accounts, transferring funds to another account, and ultimately cashing out for profit.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Below are some insights about our dataset:</b></p>
<ol style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>We have 92 fraud transactions, which account for 0.14% of the dataset.</li>
    <li>Out of these 92 fraud transactions, 47 are of type TRANSFER, and 45 are of type CASH_OUT.</li>
    <li>Approximately 97.83% of our fraud transactions have a transaction amount equal to oldbalanceOrig, indicating account cleanout.</li>
    <li>About 71.74% of our fraud transactions have the recipient's old balance as zero.</li>
    <li>The isFlaggedFraud indicator is correct only two times among our 92 fraud transactions.</li>
</ol>


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Clean up the dataset</b>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Based on what we discovered above, we will:</p>
<ol style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>Remove all data that isn't 'CASH OUT' or 'TRANSFER'.</li>
    <li>Drop "nameOrig" and "nameDest" since our origin and destination accounts don't matter.</li>
    <li>Drop "isFlaggedFraud" because we have just flagged two transactions. Hence, it doesn't have much significance.</li>
</ol>

In [ ]:
clean_data = txn_data.loc[(txn_data.type == 'CASH_OUT') | (txn_data.type == 'TRANSFER')]
clean_data.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, we have reduced our dataset to 27k records.</p>

In [ ]:
clean_data = clean_data.drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)
clean_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we'll copy this Teradata Dataframe to a separate table in Vantage.</p>

In [ ]:
# Create the source data table in the database
clean_data.to_sql('clean_data', if_exists = 'replace', primary_index = 'txn_id')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Create training and testing datasets in Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We'll create two datasets for training and testing in the ratio of 80:20.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
                                    data = DataFrame('clean_data'),
                                    id_column = "txn_id",
                                    train_size = 0.80,
                                    test_size = 0.20,
                                    seed = 42
)

clean_data_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
clean_data_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

In [ ]:
print("Training Set = " + str(clean_data_train.shape[0]) + ". Testing Set = " + str(clean_data_test.shape[0]))

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Create a PMML pipeline locally using sklearn and train it</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will use Logistic Regression as our classifier model in the PMML pipeline. We leverage the Predictive Model Markup Language (PMML), which is an XML-based language that provides a way for us to define machine learning, statistical, and data mining models and to share models between PMML-compliant applications.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Create a local pandas dataframe.</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We export data outside Vantage and train the model using sklearn. We do this to simulate model training outside Vantage. Later, we save the trained model, which is in PMML format, into Vantage, and we perform in-database scoring at scale.</p>

In [ ]:
clean_data_train_df = clean_data_train.to_pandas(all_rows = True)
clean_data_train_df.reset_index(inplace = True)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Train the Pipeline and save the model locally</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We would save the model in a file called <b>mm_fraud_glm_model.pmml</b>.</p>

In [ ]:
warnings.filterwarnings("ignore")

y_train = clean_data_train_df[['isFraud']].astype(int)
X_train = clean_data_train_df[['step','amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig']].astype(float)


pipeline = PMMLPipeline([
     ("classifier", LogisticRegression())
     ])
pipeline.fit(X_train, y_train)

sklearn2pmml(pipeline, "mm_fraud_glm_model.pmml", with_repr = True)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Load trained PMML Model into Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We pass the local PMML file to the save_byom function. This function loads the <b>mm_fraud_glm_model.pmml</b> file into Vantage in a table called <b>mm_glm</b>. If there is already a model with the same name, we will delete the previous one, and a new one will be loaded.</p>

In [ ]:
# Load the PMML file into Vantage
try:
    res = save_byom(model_id = 'mm_glm1', model_file = 'mm_fraud_glm_model.pmml', table_name = 'mm_glm')

except Exception as e:
    # if our model exists, delete and rewrite
    if str(e.args).find('TDML_2200') >= 1:
        res = delete_byom(model_id = 'mm_glm1', table_name = 'mm_glm')
        res = save_byom(model_id = 'mm_glm1', model_file = 'mm_fraud_glm_model.pmml', table_name = 'mm_glm')
        pass
    else:
        raise

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Score the model directly in Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use PMMLPredict to score the model on data residing in Vantage without moving data.</p>

In [ ]:
model_tdf = retrieve_byom("mm_glm1", table_name = 'mm_glm')

# Run the PMMLPredict function in Vantage
result = PMMLPredict(
            modeldata = model_tdf,
            newdata = clean_data_test,
            accumulate = ['step','amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig', 'isFraud'],
            overwrite_cached_models = '*',
            model_output_fields = ['probability(1)', 'probability(0)']
            )

In [ ]:
result.result

<p style='font-size:16px;font-family:Arial;color:#00233C'>In our output above, we show probability(1), indicating a transaction being fraud, and probability(0), indicating a transaction being genuine.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>We observe that the value in the prediction field of the result column in json_report appears only if we have a prediction field in the result column. The prediction output column could be empty if the scoring performed with a classification model does not return a predicted value.
    <br>
    <br>
Read more about it <a href = 'https://docs.teradata.com/search/all?query=PMMLPredict+Output&content-lang=en-US'>here</a></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>8. Visualize the results (ROC curve and AUC)</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create the ROC curve, which is a graph between TPR (True Positive Rate) and FPR (False Positive Rate). We use the area under the ROC curve as a metric to evaluate how well our model can distinguish between positive and negative classes. A higher AUC indicates better performance in distinguishing between the positive and negative categories. We generally consider an AUC above 0.75 as decent.</p>

In [ ]:
from teradataml import ROC

roc_out = ROC(probability_column='"probability(1)"',
              observation_column="isFraud",
              positive_class="1",
              data=result.result,
              num_thresholds = 300)

roc_out.output_data

In [ ]:
roc_out.result

In [ ]:
# Assigning new index column
roc_out.result = roc_out.result.assign(row = 1)
# Changing the index label.
roc_out.result._index_label = ["row"]
auc = roc_out.result.get_values()[0][0]

figure = Figure(width=500, height=400, heading="Receiver Operating Characteristic (ROC) Curve")

plot = roc_out.output_data.plot(
    x=roc_out.output_data.fpr,
    y=roc_out.output_data.tpr,
    xlabel='False Positive Rate',
    ylabel='True Positive Rate',
    color='carolina blue',
    figure=figure,
    legend=f'AUC = {round(auc, 4)}',
    legend_style='lower right',
    grid_linestyle='--',
    grid_linewidth=0.5
)

plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Looking at the above ROC Curve, we can confidently say that our model has performed well on testing data. The AUC value is way above 0.75 and resonates with our understanding that the model is performing well.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demonstration, we have illustrated a simplified - but complete - overview of how we can improve a typical machine learning workflow using Vantage in conjunction with open-source tools and techniques. This combination allows us to leverage open-source innovation with Vantage's operational scale, power, and stability.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>9. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
db_drop_table(table_name = 'mm_glm')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('demo_glm_fraud');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Industry:</b> Finance</li>
    <li><b>Functionality:</b> Machine Learning</li>
    <li><b>Use Case:</b> Fraud Detection</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><a href='https://www.teradata.com/Blogs/Fraud-Busting-AI'>Fraud-Busting-AI</a></li>
    <li><a href='https://www.teradata.com/Industries/Financial-Services'>Financial Services</a></li>
    <li><a href='https://www.teradata.com/Resources/Datasheets/Move-from-Detection-to-Prevention-and-Outsmart-Fraudsters'>Move from Detection to Prevention and Outsmart Tech-Savvy Fraudsters</a></li>
</ul>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Dataset:</b>

- `txn_id`: transaction id
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (31 days simulation).
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: amount of the transaction in local currency
- `nameOrig`: customer who started the transaction
- `oldbalanceOrig`: customer's balance before the transaction
- `newbalanceOrig`: customer's balance after the transaction
- `nameDest`: customer who is the recipient of the transaction
- `oldbalanceDest`: recipient's balance before the transaction
- `newbalanceDest`: recipient's balance after the transaction
- `isFraud`: identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: flags illegal attempts to transfer more than 200,000 in a single transaction

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Uses a dataset and feature discovery methods outlined here: <a href = 'https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook'>https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook</a></li>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>